In [1]:
# sc.list_packages()

sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1667620434015_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



You are using pip version 9.0.1, however version 22.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 9.0.1, however version 22.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [2]:
confs = ["spark.yarn.executor.memoryOverhead","spark.sql.shuffle.partitions"]
print(sc._conf.get('spark.driver.memory'))
print(sc._conf.get('spark.app.name'))
print(sc._conf.get('spark.submit.deployMode'))
print(sc._conf.get("spark.yarn.executor.memoryOverhead"))
print(sc._conf.get("spark.sql.files.maxPartitionBytes"))
for conf in confs:
    print(conf,":",sc._conf.get(conf))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2048M
livy-session-0
client
None
None
spark.yarn.executor.memoryOverhead : None
spark.sql.shuffle.partitions : None

# Imports

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth, to_date
import boto3
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, MapType, IntegerType
from pyspark.sql.types import *
from pyspark.sql import functions as F
from collections import Counter
import pyspark
# import s3fs
# import boto3
#How to pip install while creation of cluster

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
for year in range(2015,2016):
    tone_file_path = f"s3://shrivats-dev/Tasks/WorldPrioGridTones/Daily/AllThemes/{year}/*.csv"
    df_tone =  spark.read.option("header",True).csv(tone_file_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_tone.count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2876104

In [6]:
df_tone.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+--------+-------------------+
|year|month|day|GRID_IDs|           tone_avg|
+----+-----+---+--------+-------------------+
|2015|    9|  4|  117297|-0.0636942675159236|
|2015|   11| 25|  143635|   1.70648464163823|
|2015|    6|  6|  147319|   3.25925925925926|
|2015|    8| 29|  191883|   2.04545454545454|
|2015|   10| 19|  185911|-0.9322616726898288|
+----+-----+---+--------+-------------------+
only showing top 5 rows

### Read Acled Data

In [62]:
from pyspark.sql.functions import year, month, dayofmonth, to_date
df_acled = spark.read.option("header",True).csv("s3://shrivats-dev/Data/ACLED/*.csv").drop("year")
df_acled_date = df_acled.select("*",to_date(df_acled.event_date,"dd MMMM yyyy").alias("date"))
df_acled_date = df_acled_date.withColumn("day",dayofmonth(df_acled_date.date))\
    .withColumn("month",month(df_acled_date.date))\
    .withColumn("year",year(df_acled_date.date))\
    .withColumn("GRID_IDs", df_acled_date['PRIO GRID ID']).drop('PRIO GRID ID')
# df_acled_date.show(5)
print(df_acled_date.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1430515

In [64]:
req_cols = ["day","month","year","GRID_IDs"]
df_acled_date_group_counts = df_acled_date.select(*req_cols).groupBy(*req_cols).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Joining GDELT Tone and ACLED data

In [65]:
df_acled_date_group_counts.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- GRID_IDs: string (nullable = true)
 |-- count: long (nullable = false)

In [66]:
for col_name in df_tone.columns:
    df_tone = df_tone.withColumn(col_name, df_tone[col_name].cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
for col_name in df_acled_date_group_counts.columns:
    df_acled_date_group_counts = df_acled_date_group_counts.withColumn(col_name, df_acled_date_group_counts[col_name].cast("double"))
df_acled_date_group_counts.printSchema()
df_acled_date_group_counts.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- day: double (nullable = true)
 |-- month: double (nullable = true)
 |-- year: double (nullable = true)
 |-- GRID_IDs: double (nullable = true)
 |-- count: double (nullable = false)

+----+-----+------+--------+-----+
| day|month|  year|GRID_IDs|count|
+----+-----+------+--------+-----+
|12.0|  5.0|2022.0|181360.0|  2.0|
| 9.0|  5.0|2022.0|145800.0|  1.0|
| 9.0|  5.0|2022.0|179722.0|  1.0|
| 5.0|  5.0|2022.0|191875.0|  1.0|
| 1.0|  5.0|2022.0|163986.0|  1.0|
+----+-----+------+--------+-----+
only showing top 5 rows

In [125]:
for year in range(2015,2023):
    tone_file_path = f"s3://shrivats-dev/Tasks/WorldPrioGridTones/Daily/AllThemes/{year}/*.csv"
    df_tone =  spark.read.option("header",True).csv(tone_file_path)
    df_acled_tone = df_tone.join(df_acled_date_group_counts, [*req_cols] ,"left").fillna(0, subset=['count']).na.drop()
    print("Count:", df_acled_tone.count())
    print("Distinct Prio Grids:", df_acled_tone.select("GRID_IDs").distinct().count())
    df_acled_tone = df_acled_tone.withColumn("Acled Event",df_acled_tone['count']).drop("count")
    df_acled_tone.repartition(1).write.option("header",True) \
           .csv(f"s3://shrivats-dev/Tasks/GDelt_PrioGrid_Acled_Events/{year}")
    print("="*50,"\n")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Count: 2851809
Distinct Prio Grids: 35998

Count: 3260044
Distinct Prio Grids: 36648

Count: 2918212
Distinct Prio Grids: 35535

Count: 3131488
Distinct Prio Grids: 35989

Count: 2978645
Distinct Prio Grids: 35616

Count: 2577291
Distinct Prio Grids: 34411

Count: 2470864
Distinct Prio Grids: 34118

Count: 1399427
Distinct Prio Grids: 31418

In [126]:
df_acled_op =  spark.read.option("header",True).csv(f"s3://shrivats-dev/Tasks/GDelt_PrioGrid_Acled_Events/{year}")
df_acled_op.where(df_acled_op['Acled Event'].between(24,100)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+----+--------+-----------------+-----------+
|day|month|year|GRID_IDs|         tone_avg|Acled Event|
+---+-----+----+--------+-----------------+-----------+
| 21|    2|2022|  199156|-5.18041947792422|       24.0|
| 18|    2|2022|  199156|-4.38555743243243|       24.0|
| 20|    2|2022|  199156| -3.6326295900764|       27.0|
+---+-----+----+--------+-----------------+-----------+

In [127]:
df_acled_op.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+----+--------+--------------------+-----------+
|day|month|year|GRID_IDs|            tone_avg|Acled Event|
+---+-----+----+--------+--------------------+-----------+
|  1|    1|2022|   74095| -1.9749452354874062|        0.0|
|  1|    1|2022|   78005|  0.9393776542649018|        0.0|
|  1|    1|2022|   79118|  -2.868747920233297|        0.0|
|  1|    1|2022|   79189|  1.5171612357967874|        0.0|
|  1|    1|2022|   81579|-0.39616665258341255|        0.0|
|  1|    1|2022|   81957|    1.11576011157601|        0.0|
|  1|    1|2022|   89228| -1.0595781812602871|        0.0|
|  1|    1|2022|   89696|    -2.8633171258779|        0.0|
|  1|    1|2022|   92107|-0.21223171195174229|        0.0|
|  1|    1|2022|   95307|  0.3430558943774452|        0.0|
|  1|    1|2022|   96182|   -6.71140939597315|        0.0|
|  1|    1|2022|  101221|   -1.07913669064748|        0.0|
|  1|    1|2022|  101936|    2.37730061349693|        0.0|
|  1|    1|2022|  101940|  -5.836621646188215|        0.

## Summarizing Latin America Prio Grids

In [56]:
#Helper Functions

#For checking whether the two dataframes are close in number of prio grids
def isClose(num1, num2, thresh):
    diff = abs(num2-num1)
    max_num = max(num1, num2)
    
    if diff/max_num <= thresh:
        print(f"Close enough. Differ by {round((diff/max_num) * 100, 3)}%")
        return True
    else:
        print(f"Not close enough. The difference is {round((diff/max_num) * 100, 3)}%")
        return False

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_regions = spark.read.option("header",True).csv("s3://shrivats-dev/Data/ACLED/prio grid to region.csv")
req_region = "Latin America & Caribbean"
df_req_region = df_regions[df_regions.region == "Latin America & Caribbean"]
req_prio_grids = [x['priogrid'] for x in df_req_region.select("priogrid").collect()]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_gdelt_acled =  spark.read.option("header",True).csv(f"s3://shrivats-dev/Tasks/GDelt_PrioGrid_Acled_Events/*")
df_gdelt_acled.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
df_acled_req_region = df_gdelt_acled[df_gdelt_acled['GRID_IDs'].isin(req_prio_grids)]

isClose(df_acled_req_region.select("GRID_IDs").distinct().count() ,len(req_prio_grids), .3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Close enough. Differ by 26.374%
True

In [81]:
#Converting all values to float
for column in df_acled_req_region.columns:
    df_acled_req_region = df_acled_req_region.withColumn(column, df_acled_req_region[column].cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
from pyspark.sql.functions import sum,avg,max,count, col
df_acled_req_region_monthly = df_acled_req_region.groupBy("month", "year", "GRID_IDs").agg(sum("Acled Event").alias("Acled Events Count"),\
                                                             avg("tone_avg").alias("tone_avg"))\
                                                            .orderBy(col("year"), col("month"),col("GRID_IDs"))\
                                                            .withColumn("GRID_IDs", col("GRID_IDs").cast("integer"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
df_acled_req_region_monthly.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+--------+------------------+-------------------+
|month|  year|GRID_IDs|Acled Events Count|           tone_avg|
+-----+------+--------+------------------+-------------------+
|  2.0|2015.0|   49184|               0.0|                3.0|
|  2.0|2015.0|   49186|               0.0|0.14285714285714285|
|  2.0|2015.0|   50619|               0.0|                0.0|
|  2.0|2015.0|   50623|               0.0|                0.0|
|  2.0|2015.0|   50624|               0.0| 0.3333333333333333|
+-----+------+--------+------------------+-------------------+
only showing top 5 rows

In [102]:
frequency = "Monthly"
formatted_region = req_region.replace("&","and").replace(" ","_")
df_acled_req_region_monthly.repartition(1).write.option("header",True) \
           .csv(f"s3://shrivats-dev/Tasks/GDelt_PrioGrid_Acled_Event_Region_Specific/{formatted_region}/{frequency}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…